In [126]:
import pandas as pd
import numpy as np
import torch
import re
import nltk
from transformers import Trainer, TrainingArguments, AutoModel, AutoTokenizer
from tqdm import tqdm
nltk.download('words')
from nltk.corpus import words
# get name of the device
torch.cuda.get_device_name(0)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


'NVIDIA GeForce RTX 3090'

In [51]:
bert_model = 'dumitrescustefan/bert-base-romanian-cased-v1'

In [127]:
df = pd.read_csv("train.csv")

In [128]:
df

,id,title,content,class
0,0,PSD în alertă,Prăbușirea PSD de la altitudinea sigură a celo...,True
1,1,"În amintirea Vioricăi, milioane de români beau...",Moțiunea de cenzură care a doborât guvernul Dă...,True
2,2,"Dramă! Când credea că nu se poate mai rău, un ...",Credeai că ai ajuns la fundul sacului? Înseamn...,True
3,3,"Spania - România, 5-0. „Tricolorii”, îngenunch...","Echipa națională a României a fost umilită, lu...",False
4,4,"Campanie electorală, veselie generală","Toate cresc în campania electorală, cît n-au c...",True
...,...,...,...,...
70570,70570,Spații de cazare la mare înălțime. Refugiul Ie...,"Pentru drumeția de azi, un echipaj de interven...",True
70571,70571,"Sfântul Vali, colanții de la mall și miorița n...",Pe Mădălin nu l-a lăsat tac-su să termine opt ...,True
70572,70572,"Banii sănătății unde e, banii sănătății ce-ați...","Reforma sănătății a început, în România, pe vr...",True
70573,70573,Pentru că nu poate da o lege să nu mai bată vâ...,"Premierul Mihai Tudose a fost, din nou, deranj...",True


In [132]:
# count the average number of words in a title
df['title'].apply(lambda x: len(x.split())).mean()

13.973332955479355

In [137]:
df['content'].dropna().apply(lambda x: len(x.split())).mean()

332.3609261201009

In [136]:
# make the 'content' column type string
df['content'] = df['content'].astype(str)

C:\Users\Alex\AppData\Local\Temp\ipykernel_12496\1972564188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content'] = df['content'].astype(str)


In [134]:
# count the average number of words in a content, if the content is empty, the number of words is 0
df['content'].apply(lambda x: len(x.split()) if x != '' else 0).mean()

AttributeError: 'float' object has no attribute 'split'

In [131]:
# remove rows with empty title
df = df.dropna(subset=['title'])

In [18]:
df

,id,title,content,class
0,0,PSD în alertă,Prăbușirea PSD de la altitudinea sigură a celo...,True
1,1,"În amintirea Vioricăi, milioane de români beau...",Moțiunea de cenzură care a doborât guvernul Dă...,True
2,2,"Dramă! Când credea că nu se poate mai rău, un ...",Credeai că ai ajuns la fundul sacului? Înseamn...,True
3,3,"Spania - România, 5-0. „Tricolorii”, îngenunch...","Echipa națională a României a fost umilită, lu...",False
4,4,"Campanie electorală, veselie generală","Toate cresc în campania electorală, cît n-au c...",True
...,...,...,...,...
70570,70570,Spații de cazare la mare înălțime. Refugiul Ie...,"Pentru drumeția de azi, un echipaj de interven...",True
70571,70571,"Sfântul Vali, colanții de la mall și miorița n...",Pe Mădălin nu l-a lăsat tac-su să termine opt ...,True
70572,70572,"Banii sănătății unde e, banii sănătății ce-ați...","Reforma sănătății a început, în România, pe vr...",True
70573,70573,Pentru că nu poate da o lege să nu mai bată vâ...,"Premierul Mihai Tudose a fost, din nou, deranj...",True


In [42]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [112]:
# preprocess the data for the model
def replace_unwanted_characters(df):
    df['content'] = df['content'].fillna('')
    df['title'] = df['title'].fillna('')
    df['title'] = df['title'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    df['content'] = df['content'].apply(lambda x: x.replace("ţ", "ț").replace("ş", "ș").replace("Ţ", "Ț").replace("Ş", "Ș"))
    return df

repeated_characters = [x * 3 for x in "AĂÂBCDEFGHÎJKLMNOPQRSȘTȚUVWXYZaăâbcdefghîjklmnopqrsștțuvwxyz,.-/';[]-!@#$%^&*()?+"]
repeated_characters.append('iiii')
repeated_characters.append('IIII')

profanity_list = [
    "muie",
    "laba",
    "labă",
    "pula",
    "pulă",
    "pizda",
    "pizdă",
    "ce pula",
    "ce pulă",
    "ce pizda",
    "ce pizdă",
    "caca",
    "cacat",
    "căcat",
    "pipi",
    "pisat",
    "pișat",
    "pishat",
    "rahat",
    "kkt",
    "kk",
    "plm",
    "ma fut",
    "mă fut",
    "ma cac",
    "mă cac",
    "ma pis",
    "mă pis",
    "ma pish",
    "mă pish",
    "pwla",
    "pwlă",
    "p.u.l.a.",
    "poola",
    "naiba",
    "dracu",
    "draq",
    "drecu",
    "naibii",
    "dracului",
    "drecului",
    "drqlui",
    "coaie",
    "coae",
    "sloboz",
    "lindic",
    "gaoz",
    "ochiul maro",
    "floci",
    "cur",
    "futai",
    "futare",
    "futere",
    "popou",
    "nanau",
    "pulii",
    "pulii mele",
    "coaiele",
    "coaiele mele",
    "pulile",
    "măta",
    "mata",
    "mă-tii",
    "mă-ta",
    "mă-ti",
]

english_words = set(words.words())

def normalize_text(df):

    df['title'] = df['title'].apply(lambda x : re.sub(r"<.*?>", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"<.*?>", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"\.\.\.", '', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"\.\.\.", '', x))

    df['title'] = df['title'].apply(lambda x : re.sub(r"#[\w+-]+", '[HTAG]', x ))
    df['content'] = df['content'].apply(lambda x:re.sub(r"#[\w+-]+", '[HTAG]', x))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[STAR]" if "*" in word else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x:' '.join(["[STAR]" if "*" in word else word for word in x.split()]))

    df['title'] = df['title'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[REP]" if any([rep in word for rep in repeated_characters]) else word for word in x.split()]))

    pattern = r'^[!?"\',“”-]*(.*?)[!?"\',“”-]*$'
    df['title'] = df['title'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))
    df['content'] = df['content'].apply(lambda x : ' '.join(["[PROF]" if re.sub(pattern, r'\1', word) in profanity_list else word for word in x.split()]))


    emoji_pattern = r"""
                    (?:
                      [<>]?
                      [:;=8]                     # eyes
                      [\-o\*\']?                 # optional nose
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      |
                      [\)\]\(\[dDpP/\:\}\{@\|\\] # mouth
                      [\-o\*\']?                 # optional nose
                      [:;=8]                     # eyes
                      [<>]?
                      |
                      </?3                       # heart
                    )"""
    df['title'] = df['title'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))
    df['content'] = df['content'].apply(lambda x : re.sub(emoji_pattern, '[EMOJI]', x ))


    return df

In [113]:
def get_titles_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['title'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title, label in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)
        y_train.append(label)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = list(zip(df['content'].tolist(), df['class'].tolist()))

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for content, label in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
            y_train.append(label)
        else:
            X_train.append([0 for _ in range(768)])
            y_train.append(label)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)

    text_embeddings, _ = get_titles_tokens(df)
    content_embeddings, _ = get_content_tokens(df)

    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)

In [114]:
df = pd.read_csv('second.csv')
preprocess_dataset(df, 'second')

100%|██████████| 17643/17643 [52:10<00:00,  5.64it/s] 


In [115]:
df_test = pd.read_csv('test.csv')
preprocess_dataset(df_test, 'test')

KeyError: 'class'

In [116]:
df_third = pd.read_csv('third.csv')
preprocess_dataset(df_third, 'third')

100%|██████████| 17643/17643 [46:01<00:00,  6.39it/s]


In [121]:
def split_train_in_six(dataset):
    length = int(len(dataset) / 6)

    first = dataset[:length]
    second = dataset[length:2*length]
    third = dataset[2*length:3*length]
    fourth = dataset[3*length:4*length]
    fifth = dataset[4*length:5*length]
    sixth = dataset[5*length:]

    first.to_csv('first_test.csv', encoding='utf-8', index=False)
    second.to_csv('second_test.csv', encoding='utf-8', index=False)
    third.to_csv('third_test.csv', encoding='utf-8', index=False)
    fourth.to_csv('fourth_test.csv', encoding='utf-8', index=False)
    fifth.to_csv('fifth_test.csv', encoding='utf-8', index=False)
    sixth.to_csv('sixth_test.csv', encoding='utf-8', index=False)
    

split_train_in_six(pd.read_csv('test.csv'))

In [122]:
def get_titles_tokens(df):
    X_train = []
    aux = df['title'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for title in tqdm(aux):
        input_ids = torch.tensor(tokenizer.encode(title, add_special_tokens=True, max_length=32, padding='max_length', truncation=True, return_tensors="np"))
        outputs = robert(input_ids)
        mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
        X_train.append(mean_pooling)

    X_train = np.array(X_train)

    return X_train

def get_content_tokens(df):
    X_train = []
    y_train = []
    aux = df['content'].tolist()

    tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    added_toks = tokenizer.add_special_tokens({'additional_special_tokens': ['[STAR]', '[HTAG]', '[REP]', '[PROF]', '[EMOJI]']})

    robert = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-cased-v1")
    robert.resize_token_embeddings(len(tokenizer))

    for content in tqdm(aux):
        if len(content) > 0:
            input_ids = torch.tensor(tokenizer.encode(content, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors="np"))
            outputs = robert(input_ids)
            mean_pooling = torch.mean(outputs[0], dim = 1).squeeze(0).detach()
            X_train.append(mean_pooling)
        else:
            X_train.append([0 for _ in range(768)])

    X_train = np.array(X_train)

    return X_train


def preprocess_dataset(df, name):
    df = replace_unwanted_characters(df)
    df = normalize_text(df)

    text_embeddings = get_titles_tokens(df)
    content_embeddings = get_content_tokens(df)

    np.save(name+'title.npy', text_embeddings)
    np.save(name+'content.npy', content_embeddings)

In [124]:
df_fourth_test = pd.read_csv('fourth_test.csv')
preprocess_dataset(df_fourth_test, 'fourth_test')

100%|██████████| 6111/6111 [12:26<00:00,  8.19it/s]


In [125]:
df_fifth_test = pd.read_csv('fifth_test.csv')
preprocess_dataset(df_fifth_test, 'fifth_test')

100%|██████████| 6111/6111 [12:02<00:00,  8.45it/s]
